#**Model Attempt #2**

Improvments To Be Made:

Data Augmentation- RandomHorizontalFlip, RandomRotation, ColorJitter

Make Image BIGGER- 64×64 to 128×128

CNN Update- Add BatchNorm2d after conv layers

Dropout- added 0.5 before final fully connected layer

Learning Rate Scheduler- ReduceLROnPlateau

Lengthen Training and Early Stopping- stop if no improvement in 5 epochs

In [ ]:
# Load in libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
import random
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.image import imread
import os
from imageio import imread

# Stop warnings
import warnings
warnings.filterwarnings('ignore')

#from google.colab import drive
#drive.mount('/content/drive')
#!unzip /content/drive/MyDrive/dataset.zip -d /content/dataset
#path = '/content/dataset/dataset'


In [ ]:
# Load data
width = []
height = []
channel_color = []
weather_type = []

path = '/content/dataset/dataset'

# for each subfolder (category)
for folder in os.listdir(path):
    subfolder_path = os.path.join(path, folder_name)

    # skip if not a directory
    if not os.path.isdir(subfolder_path):
        continue

    for image_file in os.listdir(subfolder_path):
        image_path = os.path.join(subfolder_path, image_file)

        # skip directories inside category folders
        if os.path.isdir(image_path):
            continue

        image = imread(image_path)

        # handle grayscale images
        if len(image.shape) < 3:
            image = image.reshape(image.shape + (1,))

        height, width, channel = image.shape
        width.append(width)      # width on x-axis
        height.append(height)     # height on y-axis
        channel_color.append(channel)
        weather_type.append(folder_name)  # use folder name as label

# create DataFrame
df = pd.DataFrame({
    'width': width,
    'height': height,
    'channels': channel_color,
    'weather': weather_type
})

df.head()

Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
from collections import Counter
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Data augmentation
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # larger image size
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])


# Load dataset
dataset = datasets.ImageFolder('/content/dataset/dataset', transform=transform)
num_classes = len(dataset.classes)
print("Classes:", dataset.classes)

# Optional: check number of images per class
labels = [label for _, label in dataset]
counts = Counter(labels)
print("Images per class:", counts)

# Train/Validation split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Define CNN with BatchNorm and Dropout
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)  # adjusted for 128x128 input
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 32 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = SimpleCNN(num_classes).to(device)

# Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Learning rate scheduler (without verbose)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

# Training loop with early stopping
epochs = 20
best_val_acc = 0
best_model_wts = copy.deepcopy(model.state_dict())
patience_counter = 0
early_stop_patience = 5

for epoch in range(epochs):
    # Training
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    # Step scheduler and print current LR
    scheduler.step(avg_val_loss)
    print(f"Current LR: {optimizer.param_groups[0]['lr']}")

    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stop_patience:
            print("Early stopping triggered")
            break

# Load best weights
model.load_state_dict(best_model_wts)
print(f"Best Validation Accuracy: {best_val_acc:.2f}%")
